In [1]:
!pip install openpyxl


In [2]:
!pip install xgboost


In [3]:
!pip install shap


In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import shap
import numpy as np

In [5]:
data = pd.read_excel('app_data.xlsx')

In [6]:
data = data.iloc[:,:39]

In [7]:
data2 = data[['Age','BMI','Sex','Neutrophil_Percentage','WBC_Count','Neutrophilia','Ketones_in_Urine','Peritonitis','Lower_Right_Abd_Pain','Contralateral_Rebound_Tenderness','Ipsilateral_Rebound_Tenderness','Loss_of_Appetite','Body_Temperature','Diagnosis']]

In [8]:
data2_wo = data2.drop(columns=['Ketones_in_Urine','Ipsilateral_Rebound_Tenderness'])

In [9]:
data_wo_clean = data2_wo.dropna()

In [10]:
diag_map = {'appendicitis':1,'no appendicitis':0}
data_wo_clean['Diagnosis'] = data_wo_clean['Diagnosis'].map(diag_map)

/var/folders/q_/2y0lpyw55zl65z43mxhtc48w0000gn/T/ipykernel_93482/1016667334.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_wo_clean['Diagnosis'] = data_wo_clean['Diagnosis'].map(diag_map)


In [11]:
X_wo = data_wo_clean.drop(columns='Diagnosis')
y_wo = data_wo_clean['Diagnosis']

y_wo = y_wo.astype(int)

In [12]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
num_feature = ['Age', 'BMI', 'Neutrophil_Percentage',
    'WBC_Count', 'Body_Temperature']
cat_feature = ['Sex', 'Neutrophilia',
    'Peritonitis', 'Lower_Right_Abd_Pain',
    'Contralateral_Rebound_Tenderness',
    'Loss_of_Appetite']
preprocessor = ColumnTransformer([
    ('num',StandardScaler(),num_feature),
    ('cat',OneHotEncoder(drop='first',handle_unknown='ignore'),cat_feature)
],remainder='drop')

In [13]:
from sklearn.model_selection import train_test_split
X_wo_train, X_wo_val, y_wo_train, y_wo_val = train_test_split(X_wo,y_wo,test_size=0.2,random_state=42,stratify=y_wo)

In [14]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_curve, roc_auc_score
import matplotlib.pyplot as plt
import seaborn as sns
import joblib

final_rf_pipeline = Pipeline([
    ("preprocessor", preprocessor),
    ("model", RandomForestClassifier(n_estimators=200, max_depth=10, random_state=42))
])


final_rf_pipeline.fit(X_wo_train, y_wo_train)


y_pred = final_rf_pipeline.predict(X_wo_val)
y_proba = final_rf_pipeline.predict_proba(X_wo_val)[:, 1]